In [1]:
#download data
try:
    import urllib.request as urllib2
except ImportError:
    import urllib2
csv_file = urllib2.urlopen("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv")
with open('dpc-covid19-ita-regioni.csv','wb') as output:
  output.write(csv_file.read())

In [2]:
local_path='/home/rosario/Codice/kubeflow-test/train/src/'
#local_path='/opt/ml/code'

In [10]:

import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf


In [4]:
# load dataset
dataframe = pandas.read_csv(local_path+'dpc-covid19-ita-regioni.csv')
dataset = dataframe.values
dataframe.head()

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_attualmente_positivi,nuovi_attualmente_positivi,dimessi_guariti,deceduti,totale_casi,tamponi
0,2020-02-24 18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,0,0,0,0,0,5
1,2020-02-24 18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,0,0,0,0,0,0
2,2020-02-24 18:00:00,ITA,4,P.A. Bolzano,46.499335,11.356624,0,0,0,0,0,0,0,0,0,1
3,2020-02-24 18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,0,0,0,0,0,1
4,2020-02-24 18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,0,0,0,0,0,10


In [5]:
X = dataframe.drop(columns=['totale_casi']).drop(columns=['data']).drop(columns=['stato']).drop(columns=['denominazione_regione'])
X.head()

,codice_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_attualmente_positivi,nuovi_attualmente_positivi,dimessi_guariti,deceduti,tamponi
0,13,42.351222,13.398438,0,0,0,0,0,0,0,0,5
1,17,40.639471,15.805148,0,0,0,0,0,0,0,0,0
2,4,46.499335,11.356624,0,0,0,0,0,0,0,0,1
3,18,38.905976,16.594402,0,0,0,0,0,0,0,0,1
4,15,40.839566,14.250850,0,0,0,0,0,0,0,0,10


In [6]:
X.shape

(378, 12)

In [7]:
Y = dataframe[['totale_casi']]
Y.head()


,totale_casi
0,0
1,0
2,0
3,0
4,0


In [8]:
from keras.utils import to_categorical
Y_categorical=to_categorical(Y)
Y_categorical

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [11]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(512, activation='relu', input_dim=X.shape[1]))
model.add(tf.keras.layers.Dense(256, activation='relu')) 
model.add(tf.keras.layers.Dense(128, activation='relu')) 
model.add(tf.keras.layers.Dense(Y_categorical.shape[1], activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [13]:
model.fit(X, Y_categorical, epochs=10)

Epoch 1/10
378/378 [==============================] - 1s 2ms/sample - loss: nan - acc: 0.0820                            
Epoch 2/10
378/378 [==============================] - 1s 2ms/sample - loss: nan - acc: 0.1931
Epoch 3/10
378/378 [==============================] - 0s 1ms/sample - loss: nan - acc: 0.1931
Epoch 4/10
378/378 [==============================] - 1s 2ms/sample - loss: nan - acc: 0.1931
Epoch 5/10
378/378 [==============================] - 1s 3ms/sample - loss: nan - acc: 0.1931
Epoch 6/10
378/378 [==============================] - 1s 3ms/sample - loss: nan - acc: 0.1931
Epoch 7/10
378/378 [==============================] - 1s 2ms/sample - loss: nan - acc: 0.1931
Epoch 8/10
378/378 [==============================] - 0s 1ms/sample - loss: nan - acc: 0.1931
Epoch 9/10
378/378 [==============================] - 0s 1ms/sample - loss: nan - acc: 0.1931
Epoch 10/10
378/378 [==============================] - 1s 1ms/sample - loss: nan - acc: 0.1931


In [15]:
loss, acc = model.evaluate(X, Y_categorical) #creare il test dataset!!
print("Loss sul test set: %.4f" % loss)
print("Accuracy sul test set: %.4f" % acc)

336/336 [==============================] - 0s 921us/step
Loss sul test set: nan
Accuracy sul test set: 0.2173


In [36]:
import tensorflow as tf
saved_to_path = tf.contrib.saved_model.save_keras_model(model, local_path+'model')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in expo

In [17]:
from tensorflow.python.lib.io import file_io

MODEL_FILE = 'keras_saved_model.h5'
output_model_path='/home/rosario/Codice/kubeflow-test/train/src/keras_model' #from args

print('saved model to ', output_model_path)
model.save(MODEL_FILE)
with file_io.FileIO(MODEL_FILE, mode='rb') as input_f:
  with file_io.FileIO(output_model_path + '/' + MODEL_FILE, mode='wb+') as output_f:
    output_f.write(input_f.read())

saved model to  /home/rosario/Codice/kubeflow-test/train/src/keras_model


In [ ]:
Path(output_model_path_file).parent.mkdir(parents=True, exist_ok=True)
Path(output_model_path_file).write_text(args.output_model_path)

In [13]:
#Create service account with permissions to read/write on Cloud storage bucket and get JSON
import platform
import os

platform=platform.system()
print(platform)
if platform == 'Windows':
    os.environ['GOOGLE_APPLICATION_CREDENTIALS']=r'C:\Users\lafacero\Downloads\ai-vqc-fb43ef70f5ec.json'
elif platform == 'Linux':
    os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/home/rosario/Codice/kubeflow-test/train/auth/ai-vqc-fb43ef70f5ec.json'
print(os.getenv('GOOGLE_APPLICATION_CREDENTIALS'))

Linux
/home/rosario/Codice/kubeflow-test/train/auth/ai-vqc-fb43ef70f5ec.json


In [16]:
import os
import glob
from google.cloud import storage

PROJECT='ai-vqc'
BUCKET='ai-vqc-kubeflow-output'
PREFIX=''
FILENAME=local_path+'/model/'

def copy_local_directory_to_gcs(local_path, bucket_name, gcs_path):
    """Recursively copy a directory of files to GCS.

    local_path should be a directory and not have a trailing slash.
    """
    assert os.path.isdir(local_path)
    for local_file in glob.glob(local_path + '/**'):
        if not os.path.isfile(local_file):
            continue
        storage_client = storage.Client()
        bucket = storage_client.get_bucket(bucket_name)
        remote_path = os.path.join(gcs_path, local_file[1 + len(local_path)-1 :])
        blob = bucket.blob(remote_path)
        blob.upload_from_filename(local_file)

copy_local_directory_to_gcs(local_path+'/model/', BUCKET, 'model')